In [1]:

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf

In [2]:
data = pd.read_csv("lap_times.csv")

scaler = MinMaxScaler(feature_range=(0, 1))
data['lap_times_normalized'] = scaler.fit_transform(data['lap_time'].values.reshape(-1, 1))

sequence_length = 3  

X, y = [], []
for i in range(len(data) - sequence_length):
    X.append(data['lap_times_normalized'].values[i:i+sequence_length])
    y.append(data['lap_times_normalized'].values[i+sequence_length])

X = np.array(X)
y = np.array(y)


X = X.reshape(X.shape[0], sequence_length, 1)


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


model = tf.keras.Sequential([
    tf.keras.layers.LSTM(50, input_shape=(sequence_length, 1)),
    tf.keras.layers.Dense(1)
])


model.compile(optimizer='adam', loss='mean_squared_error')

In [3]:
history = model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_test, y_test), verbose=1)

loss = model.evaluate(X_test, y_test, verbose=0)
print('Test Loss:', loss)

Epoch 1/50

1/1 [==============================] - 3s 3s/step - loss: 0.0689 - val_loss: 0.5579
Epoch 2/50
1/1 [==============================] - 0s 36ms/step - loss: 0.0648 - val_loss: 0.5396
Epoch 3/50
1/1 [==============================] - 0s 37ms/step - loss: 0.0609 - val_loss: 0.5216
Epoch 4/50
1/1 [==============================] - 0s 40ms/step - loss: 0.0574 - val_loss: 0.5039
Epoch 5/50
1/1 [==============================] - 0s 83ms/step - loss: 0.0541 - val_loss: 0.4867
Epoch 6/50
1/1 [==============================] - 0s 32ms/step - loss: 0.0510 - val_loss: 0.4697
Epoch 7/50
1/1 [==============================] - 0s 32ms/step - loss: 0.0483 - val_loss: 0.4532
Epoch 8/50
1/1 [==============================] - 0s 32ms/step - loss: 0.0458 - val_loss: 0.4371
Epoch 9/50
1/1 [==============================] - 0s 32ms/step - loss: 0.0436 - val_loss: 0.4215
Epoch 10/50
1/1 [==============================] - 0s 33ms/step - loss: 0.0417 - val_loss: 0.4064
Epoch 11/50
1/1 [=============

In [4]:
with open('lap_times.csv', "r", encoding="utf-8", errors="ignore") as scraped:
    final_line = scraped.readlines()[-1]

final_line_arr = final_line.split(',')
last_lap_count = int(final_line_arr[1])

new_lap_number = last_lap_count + 1


new_lap_number_normalized = scaler.transform([[new_lap_number]])


input_sequence = []
for i in range(sequence_length, 0, -1):
    input_sequence.append(new_lap_number_normalized - i)


input_sequence = np.array(input_sequence).reshape(1, sequence_length, 1)


predicted_lap_time_normalized = model.predict(input_sequence)

predicted_lap_time = scaler.inverse_transform(predicted_lap_time_normalized)

print("Predicted lap time for lap number", new_lap_number, ":", predicted_lap_time[0][0])


1/1 [==============================] - 0s 391ms/step
Predicted lap time for lap number 14 : 91.9972
